In [10]:
import numpy as np
from PIL import Image
import os
import csv
import matplotlib.pyplot as plt

In [2]:
def calculate_black_mass(image_array):
    height, width = image_array.shape
    half_height, half_width = height // 2, width // 2

    quarters = [
        image_array[:half_height, :half_width],    # Верхняя левая четверть
        image_array[:half_height, half_width:],    # Верхняя правая четверть
        image_array[half_height:, :half_width],    # Нижняя левая четверть
        image_array[half_height:, half_width:]     # Нижняя правая четверть
    ]

    black_masses = [np.sum(quarter == 0) for quarter in quarters]
    return black_masses

In [3]:
def calculate_normalized_black_mass(image_array, black_masses):
    height, width = image_array.shape
    quarter_area = (height // 2) * (width // 2)
    normalized_black_masses = [mass / quarter_area for mass in black_masses]
    return normalized_black_masses

In [4]:
def calculate_center_of_gravity(image_array):
    black_pixels = np.argwhere(image_array == 0)
    if len(black_pixels) == 0:
        return (0, 0)
    
    center_of_gravity = np.mean(black_pixels, axis=0)
    return tuple(center_of_gravity)

In [5]:
def calculate_normalized_center_of_gravity(image_array, center_of_gravity):
    height, width = image_array.shape
    normalized_center_of_gravity = (
        center_of_gravity[0] / height,
        center_of_gravity[1] / width
    )
    return normalized_center_of_gravity

In [6]:
def calculate_moments_of_inertia(image_array, center_of_gravity):
    height, width = image_array.shape
    y_center, x_center = center_of_gravity

    y_indices, x_indices = np.indices((height, width))
    y_indices = y_indices - y_center
    x_indices = x_indices - x_center

    horizontal_moment = np.sum((image_array == 0) * (x_indices ** 2))
    vertical_moment = np.sum((image_array == 0) * (y_indices ** 2))

    return horizontal_moment, vertical_moment

In [7]:
def calculate_normalized_moments_of_inertia(image_array, moments_of_inertia):
    height, width = image_array.shape
    area = height * width

    normalized_horizontal_moment = moments_of_inertia[0] / area
    normalized_vertical_moment = moments_of_inertia[1] / area

    return normalized_horizontal_moment, normalized_vertical_moment

In [8]:
def calculate_profiles(image_array):
    profile_x = np.sum(image_array == 0, axis=0)
    profile_y = np.sum(image_array == 0, axis=1)
    return profile_x, profile_y

In [17]:
def save_profiles(profile, filename):
    plt.figure()
    plt.bar(range(len(profile)), profile)
    plt.xlabel('Index')
    plt.ylabel('Count')
    plt.savefig(filename)
    plt.close()

In [19]:
def process_images_in_folder(folder_path, csv_path, profile_folder):
    features = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
            file_path = os.path.join(folder_path, filename)
            image = Image.open(file_path).convert('L')
            image_array = np.array(image)

            black_masses = calculate_black_mass(image_array)
            normalized_black_masses = calculate_normalized_black_mass(image_array, black_masses)
            center_of_gravity = calculate_center_of_gravity(image_array)
            normalized_center_of_gravity = calculate_normalized_center_of_gravity(image_array, center_of_gravity)
            moments_of_inertia = calculate_moments_of_inertia(image_array, center_of_gravity)
            normalized_moments_of_inertia = calculate_normalized_moments_of_inertia(image_array, moments_of_inertia)
            profile_x, profile_y = calculate_profiles(image_array)

            features.append({
                "filename": filename,
                "black_masses": black_masses,
                "normalized_black_masses": normalized_black_masses,
                "center_of_gravity": center_of_gravity,
                "normalized_center_of_gravity": normalized_center_of_gravity,
                "moments_of_inertia": moments_of_inertia,
                "normalized_moments_of_inertia": normalized_moments_of_inertia,
                "profile_x": profile_x.tolist(),
                "profile_y": profile_y.tolist()
            })
        profile_x_path = os.path.join(profile_folder, f"{filename}_profile_x.png")
        profile_y_path = os.path.join(profile_folder, f"{filename}_profile_y.png")
        save_profiles(profile_x, profile_x_path)
        save_profiles(profile_y, profile_y_path)
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            'filename',
            'black_mass_quarter_1', 'black_mass_quarter_2', 'black_mass_quarter_3', 'black_mass_quarter_4',
            'normalized_black_mass_quarter_1', 'normalized_black_mass_quarter_2', 'normalized_black_mass_quarter_3', 'normalized_black_mass_quarter_4',
            'center_of_gravity_x', 'center_of_gravity_y',
            'normalized_center_of_gravity_x', 'normalized_center_of_gravity_y',
            'horizontal_moment_of_inertia', 'vertical_moment_of_inertia',
            'normalized_horizontal_moment_of_inertia', 'normalized_vertical_moment_of_inertia'
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
        writer.writeheader()
        for feature in features:
            row = {
                'filename': feature['filename'],
                'black_mass_quarter_1': feature['black_masses'][0],
                'black_mass_quarter_2': feature['black_masses'][1],
                'black_mass_quarter_3': feature['black_masses'][2],
                'black_mass_quarter_4': feature['black_masses'][3],
                'normalized_black_mass_quarter_1': feature['normalized_black_masses'][0],
                'normalized_black_mass_quarter_2': feature['normalized_black_masses'][1],
                'normalized_black_mass_quarter_3': feature['normalized_black_masses'][2],
                'normalized_black_mass_quarter_4': feature['normalized_black_masses'][3],
                'center_of_gravity_x': feature['center_of_gravity'][0],
                'center_of_gravity_y': feature['center_of_gravity'][1],
                'normalized_center_of_gravity_x': feature['normalized_center_of_gravity'][0],
                'normalized_center_of_gravity_y': feature['normalized_center_of_gravity'][1],
                'horizontal_moment_of_inertia': feature['moments_of_inertia'][0],
                'vertical_moment_of_inertia': feature['moments_of_inertia'][1],
                'normalized_horizontal_moment_of_inertia': feature['normalized_moments_of_inertia'][0],
                'normalized_vertical_moment_of_inertia': feature['normalized_moments_of_inertia'][1]
            }
            writer.writerow(row)
    

In [21]:
# Путь к папке с изображениями
folder_path = '../../symbols'

# Вызываем функцию обработки изображений
process_images_in_folder(folder_path,'output/output.csv','output')